In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


# 다중 로지스틱 회귀

학습 데이터를 만든다.

In [2]:
# xData → [공부시간, 과외시간]
temp = [[0, 10], [2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9], [20, 0]]  # 9행 2열
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[0, 10], [2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9], [20, 0]]
<class 'list'>
[[ 0 10]
 [ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]
 [20  0]]
<class 'numpy.ndarray'>


In [3]:
# yData → [합격여부] : 실제값
temp = [0, 0, 0, 0, 1, 1, 1, 1, 1]  # 1행 9열
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
print(type(yData))

# reshape(): numpy에서 데이터는 그대로 유지한 채 배열의 형태(차원)을 변경한다.
# 1행 9열 numpy 배열 → 9행 1열 numpy 배열
yData = np.array(temp).reshape(9, 1)
print(yData)
print(type(yData))

[0, 0, 0, 0, 1, 1, 1, 1, 1]
<class 'list'>
[0 0 0 0 1 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 numpy 배열을 대입하는 경우 shape 속성을 이용해서 대입되는 numpy 배열의 차원을 지정해야 한다.
# [None, 2] → placeholder에 대입되는 numpy 배열의 행의 개수는 상관없고 열의 개수는 반드시 2개라는 의미이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2])  # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1])  # yData가 대입될 placeholder

가중치 w와 편향 b를 임의의 값으로 정한다.

In [5]:
w = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32)  # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(f'w = {sess.run(w)}\nb = {sess.run(b)}')

w = [[0.9651872 ]
 [0.12623322]]
b = [0.08996475]


2023-02-02 14:28:23.057136: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-02-02 14:28:23.057712: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# 시그모이드 방정식, 오차 함수, 경사 하강법

In [6]:
# sigmoid(): 시그모이드 방정식을 계산한다.
# matmul(): 행렬을 내적한다.
y = tf.sigmoid(tf.matmul(X, w) + b)  # 시그모이드 방정식
loss = -tf.reduce_mean(
    Y * tf.log(y) + (1 - Y) * tf.log(1 - y)
)
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  # 경사 하강법

sigmoid() 메서드의 실행결과(y)가 0.5 이상이면 1을, 0.5 미만이면 -1을 리턴시킨다.

In [7]:
sess = tf.Session()
# cast(x, dtype): x에 저장된 데이터를 dtype으로 형변환한다.
# predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
# print(sess.run(predict))  # [1 2]
# predict = tf.cast([True, False], dtype=tf.int32)
# print(sess.run(predict))
predict = tf.cast([0.4 >= 0.5, 0.5 >= 0.5], dtype=tf.float32)
print(sess.run(predict))

[0. 1.]


sigmoid() 메서드를 실행한 예측값(0 또는 1)을 계산한다.

In [8]:
predict = tf.cast(y >= 0.5, dtype=tf.float32)  # 예측값, sigmoid() 메서드를 실행한 결과를 0 또는 1로 변환한다.

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [9]:
# sess = tf.Session()
# equal(): 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
# print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0)))   # True False
# equal() 메서드로 예측값과 실제값이 같은지 비교 후 True 또는 False를 1 또는 0으로 cast() 하고 전체 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.int32))

학습시킨다.

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(20001):
    # sess.run([실행 리스트], feed_dict={X: 입력 데이터, Y: 입력 데이터에 따른 실젯값})
    w_, b_, loss_, _ = sess.run([w, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if epoch % 2000 == 0:
        print(f'epoch = {epoch:5d}, w1 = {w_[0][0]:8.5f}, w2 = {w_[1][0]:8.5f}, b = {b_[0]:9.5f}, loss = {loss_:7.5f}')

epoch =     0, w1 =  0.20380, w2 =  0.51095, b =   0.50667, loss = 2.32280
epoch =  2000, w1 =  0.83333, w2 =  0.19551, b =  -6.54253, loss = 0.08538
epoch =  4000, w1 =  1.00365, w2 =  0.43378, b =  -8.98303, loss = 0.05441
epoch =  6000, w1 =  1.12726, w2 =  0.57882, b = -10.59497, loss = 0.04112
epoch =  8000, w1 =  1.22465, w2 =  0.68715, b = -11.82979, loss = 0.03336
epoch = 10000, w1 =  1.30548, w2 =  0.77471, b = -12.84051, loss = 0.02817
epoch = 12000, w1 =  1.37478, w2 =  0.84860, b = -13.69997, loss = 0.02441
epoch = 14000, w1 =  1.43555, w2 =  0.91270, b = -14.44937, loss = 0.02156
epoch = 16000, w1 =  1.48970, w2 =  0.96940, b = -15.11461, loss = 0.01932
epoch = 18000, w1 =  1.53857, w2 =  1.02027, b = -15.71317, loss = 0.01750
epoch = 20000, w1 =  1.58311, w2 =  1.06644, b = -16.25747, loss = 0.01600


테스트

In [11]:
# 테스트 데이터를 만든다.
# new_x = np.array([6, 5])    # 테스트 데이터가 1차원 numpy 배열로 만들어지기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x)  # 1차원
# print(type(new_x))
# reshape() 메서드를 이용해서 2행 1열을 1행 2열인 numpy 배열로 만들어야 한다.
new_x = np.array([100000, 0]).reshape(1, 2)
print(new_x)  # 2차원

# sigmoid() 메서드의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predict, y], feed_dict={X: new_x})
print(f'공부시간: {new_x[:, 0][0]}, 과외시간: {new_x[:, 1][0]}, 합격여부: {"합격" if result[0][0] == 1 else "불합격":3s}, 합격률: {new_y[0][0] * 100:.2f}%')

sess.close()

[[100000      0]]
공부시간: 100000, 과외시간: 0, 합격여부: 불합격, 합격률: nan%
